In [1]:
import sys 
import os
sys.path.append("../eyeprep/utils")
from eyetrack_utils import (
    load_event_files,              # Load EyeLink event files
    extract_data,                  # Extract data from raw files
    blinkrm_pupil_off,             # Remove blinks with direct pupil detection
    blinkrm_pupil_off_smooth,      # Remove blinks with pupil detection and gaussian kernel 
    interpol_nans,                 # Interpolate missing values
    detrending,                    # Remove trends from signal
    downsample_to_targetrate,      # Change sampling rate
    moving_average_smoothing,      # Smooth with moving average
    gaussian_smoothing,            # Smooth with Gaussian kernel
    extract_eye_data_and_triggers, # Extract gaze and trial timing
    convert_to_dva                 # Convert to degrees of visual angle
)

from settings_utils import load_settings        # Load yml settings function 

from plot_utils import interactive_preproc_plot # Load plotting function for visualization 

import json
import numpy as np
import h5py
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
# Define experiment parameters and load configuration settings


EYEPREP_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
main_dir = os.path.abspath(os.getcwd())
project_dir = "data"
subject = "sub-01"
task = "SacLoc"  # Saccade localization task

# get settings 
general_settings_path = os.path.abspath(os.path.join(main_dir, "../eye-tracking.yml"))
task_settings_path = os.path.abspath(os.path.join(main_dir, f"../eye-tracking_{task}.yml"))
settings = load_settings([general_settings_path, task_settings_path])
analysis_info = settings[0]

ses = analysis_info["session"]
eye = analysis_info["eye"]


## Data Extraction

We recorded eyetracking data (.edf) and converted them to tsv.gz files accompanied by metadata json files. Now we can extract the data into readable pandas dataframes and use messages and timestamps to only look at experiment relevant data. 

### Read the data in

In [3]:
def extract_event_and_physio_data(main_dir, project_dir, subject, task, ses, num_run, eye):
    """
    Extract both event timing and physiological data from raw eye tracking files.
    
    Parameters:
    -----------
    main_dir : str
        Root directory containing all study data
    project_dir : str
        Project-specific subdirectory
    subject : str
        Subject ID (e.g., 'sub-01')
    task : str
        Task name (e.g., 'SacLoc' for saccade localization)
    ses : str
        Session ID (e.g., 'ses-01')
    num_run : int
        Number of runs to load
    eye : str
        Which eye to analyze ('eye1' or 'eye2')
    
    Returns:
    --------
    df_event_runs : list of DataFrames
        Event timing data (trial onsets, offsets, etc.)
    df_data_runs : list of DataFrames
        Raw physiological data (gaze position, pupil size, timestamps)
    """
    df_event_runs = extract_data(main_dir, project_dir, subject, task, ses, num_run, eye, file_type="physioevents") # check utils to find this function! 
    df_data_runs = extract_data(main_dir, project_dir, subject, task, ses, num_run, eye, file_type="physio")
    return df_event_runs, df_data_runs

In [ ]:
df_event_runs, df_data_runs = extract_event_and_physio_data(
    main_dir, project_dir, subject, task, ses, 
    analysis_info['num_run'], eye
)

print(f"Loaded {len(df_data_runs)} runs of eye tracking data")
print(f"Data is stored in a list, each run is one element. Lets look at the data from run 1:")
print(df_data_runs[0].head())



### Extract experiment timing 
Timestamps are in milliseconds relative to the eyetracker. By looping through the message data (df_event_runs) we can extract the timestamp for the first and last trials ("first trial pattern"/"last trial pattern") and use this to filter our actual data (df_data_runs). Finally we can convert the timestamps into seconds. 

In [ ]:
# Align eye tracking data with experimental trial timing

# Extract gaze data and trial trigger times
eye_data_run, time_start_eye, time_end_eye = extract_eye_data_and_triggers(
    df_event_runs[0], 
    df_data_runs[0],
    analysis_info['first_trial_pattern'], 
    analysis_info['last_trial_pattern']
)

# Convert timestamps to seconds (easier to work with than milliseconds)
eye_times = (df_data_runs[0]['timestamp'] - df_data_runs[0]['timestamp'][0]) * 1e-3
eye_times = eye_times.to_numpy()

# Convert trial onset times to seconds relative to first eye sample
trial_onsets_sec = (time_start_eye - df_data_runs[0]['timestamp'][0]) * 1e-3

print(f"Recording duration: {eye_times[-1]:.1f} seconds")
print(f"First trial onset at: {(trial_onsets_sec)}")


## First preprocessing 
Awesome. Lets get this signal clean. We will apply the first preprocessing steps. 
1. Remove Blinks 
2. Convert to dva 
3. Interpolate NaNs 
4. Normalise pupil data 

In [6]:
def remove_blinks(data, method, sampling_rate):
    """
    Remove blink artifacts from eye tracking data.
    
    Blinks cause temporary invalid gaze measurements. We remove them by either:
    - pupil_off: Replace blink periods with NaN (pupil disappears during blinks)
    - pupil_off_smooth: Replace and smooth the transition
    
    Parameters:
    -----------
    data : ndarray
        Eye tracking data array
    method : str
        Which blink removal method to use
    sampling_rate : int
        Eye tracker sampling rate (Hz, e.g., 500)
    
    Returns:
    --------
    data : ndarray
        Data with blinks removed (replaced with NaN or smoothed)
    """
    if method == 'pupil_off':
        return blinkrm_pupil_off(data, sampling_rate)
    elif method == 'pupil_off_smooth':
        return blinkrm_pupil_off_smooth(data, sampling_rate)
    else:
        print("Warning: No blink removal method specified. Returning original data.")
        return data


def drift_correction(data, method, fixation_periods):
    """
    Correct for eye tracker drift over time.
    
    Eye trackers lose calibration during recording, causing the measured gaze
    position to drift. We correct this by:
    - linear: Fit a line and remove the trend
    - median: Use median gaze position during fixations as reference
    
    Parameters:
    -----------
    data : ndarray
        Eye tracking data
    method : str
        Drift correction method ('linear' or 'median')
    fixation_periods : list
        Time indices of known fixations (for median method)
    
    Returns:
    --------
    data : ndarray
        Drift-corrected data
    """
    if method == "linear":
        return detrending(data, type='linear')
    elif method == 'median':
        fixation_median = np.median([item for row in fixation_periods for item in row])
        return np.array([elem - fixation_median for elem in data])
    else:
        print("Warning: No drift correction method specified. Returning original data.")
        return data


def interpolate_nans(data):
    """
    Fill in missing data points (NaN values).
    
    After removing blinks, we have gaps in our data. Linear interpolation
    estimates the missing values based on surrounding data points.
    
    Parameters:
    -----------
    data : ndarray
        Data with NaN values
    
    Returns:
    --------
    data : ndarray
        Data with NaN values interpolated
    """
    return interpol_nans(data)


def normalize_data(data):
    """
    Normalize pupil size to a standard range.
    
    Pupil size varies across subjects and sessions due to lighting conditions
    and individual differences. Normalization (scaling) makes data comparable.
    
    We scale to [-1, 1] so that values are centered at 0 with equal ranges
    on both sides (useful for neural networks and other ML methods).
    
    Parameters:
    -----------
    data : ndarray
        Raw pupil size data
    
    Returns:
    --------
    data : ndarray
        Normalized pupil size
    """
    print('- normalizing pupil data to [-1, 1]')
    scaler = MinMaxScaler(feature_range=(-1, 1))
    return scaler.fit_transform(data.reshape(-1, 1)).flatten()

In [ ]:
# STEP 1: Remove blinks
# Blinks cause the pupil to disappear and gaze data to be unreliable
eye_data_run_blinks = remove_blinks(
    eye_data_run, 
    analysis_info['blinks_remove'],      # Method specified in config
    analysis_info['eyetrack_sampling']   # Sampling rate (Hz)
)
print("✓ Blinks removed")


In [9]:
# Step 2: Convert to dva 
# Transform from pixel coordinates to a viewer-independent measure: Dva ~ rotation angle of the eye 
# attention: run this only once ;)
eye_data_run_dva = convert_to_dva(eye_data_run_blinks, analysis_info['center'], analysis_info['ppd'])

print("Lets have a look at our gaze on the screen now. Run the next cell to see!")


Lets have a look at our gaze on the screen now. Run the next cell to see!


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load your image
img = mpimg.imread(f'{EYEPREP_ROOT}/docs/images/{task}_design.png')  

# Create a figure with 1 row, 2 columns
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# --- Left: gaze scatter plot ---
axes[0].scatter(
    eye_data_run_dva[:,1],
    eye_data_run_dva[:,2],
    s=1,
    alpha=0.3,
    color='blue'
)
axes[0].axhline(0, linestyle='--', linewidth=1)
axes[0].axvline(0, linestyle='--', linewidth=1)
axes[0].set_aspect('equal', adjustable='box')
axes[0].set_xlabel('Horizontal gaze (dva)')
axes[0].set_ylabel('Vertical gaze (dva)')
axes[0].set_title('2D gaze distribution (dva)')

# --- Right: image ---
axes[1].imshow(img)
axes[1].axis('off')  # hide axes
axes[1].set_title('Stimulus image')

plt.tight_layout()
plt.show()



In [ ]:
# STEP 3: Interpolate NaNs for each channel
eye_data_run_x = interpolate_nans(eye_data_run_blinks[:, 1])
eye_data_run_y = interpolate_nans(eye_data_run_blinks[:, 2])
eye_data_run_p = interpolate_nans(eye_data_run_blinks[:, 3])
print("✓ Missing values interpolated")
# STEP 4: Normalise pupil data 
eye_data_run_p_normalized = normalize_data(eye_data_run_p)
print("✓ Pupil data normalized")

# Stack it all back together
eye_data_run_interpolated = np.stack((eye_data_run[:,0],
                                 eye_data_run_x, 
                                 eye_data_run_y, 
                                 eye_data_run_p_normalized), axis=1)

In [ ]:
# Final check to see what we did! 
# Play around with the plot 
interactive_preproc_plot(eye_data_run, eye_data_run_blinks, eye_data_run_interpolated, eye_data_run_p_normalized, eye_times)